In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import random
import os

In [2]:
# make deterministic
seed = 42
from mingpt.utils import set_seed
set_seed(seed)

In [3]:
# config
numEpochs = 2 # number of epochs to train the GPT+PT model
embeddingSize = 512 # the hidden dimension of the representation of both GPT and PT
numPoints=500 # number of points that we are going to receive to make a prediction about f given x and y, if you don't know then use the maximum
numVars=5 # the dimenstion of input points x, if you don't know then use the maximum
numYs=1 # the dimension of output points y = f(x), if you don't know then use the maximum
blockSize = 100 # spatial extent of the model for its context
batchSize = 128 # batch size of training data
dataDir = 'D:/Datasets/Symbolic Dataset/Datasets/FirstDataGenerator/'
dataInfo = 'XYE_{}Var_{}Points_{}EmbeddingSize'.format(numVars, numPoints, embeddingSize)
target = 'Skeleton' #'Skeleton' #'EQ'
dataFolder = "1-5Var_RandSupport_RandLength_0to3_3.1to6_100to500Points"
ckpt = 'XYE_1-5Var_100-500Points_SymbolicGPT_GPT_PT_Summation_Padding_100_MINIMIZE'
addr = './SavedModels/bestModel/' # where to save model
maxNumFiles = 30
bestLoss = None # if there is any model to load as pre-trained one

In [4]:
import math
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, data, block_size, chars, target='EQ'):
        data_size, vocab_size = len(data), len(chars)
        print('data has %d examples, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        
        # padding token
        self.paddingToken = '_'
        self.paddingID = self.stoi[self.paddingToken]
        self.stoi[self.paddingToken] = self.paddingID
        self.itos[self.paddingID] = self.paddingToken
        self.threshold = [-1,1]
        
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data # it should be a list of examples
        self.target = target
    
    def __len__(self):
        return len(self.data)-1

    def __getitem__(self, idx):
        # grab an example from the data
        chunk = self.data[idx] # sequence of tokens including x, y, eq, etc.
        
        try:
            chunk = json.loads(chunk) # convert the sequence tokens to a dictionary
        except:
            print("Couldn't convert to json: {}".format(chunk))
            
        # encode every character in the equation to an integer
        # < is SOS, > is EOS
        dix = [self.stoi[s] for s in '<'+chunk[self.target]+'>']
        inputs = dix[:-1]
        outputs = dix[1:]
        
        # add the padding to the equations
        paddingSize = max(self.block_size-len(inputs),0)
        paddingList = [self.paddingID]*paddingSize
        inputs += paddingList
        outputs += paddingList 
        
        # make sure it is not more than what should be
        inputs = inputs[:self.block_size]
        outputs = outputs[:self.block_size]
        
        # extract points from the input sequence
        points = torch.zeros(numVars+numYs, numPoints)
        for idx, xy in enumerate(zip(chunk['X'], chunk['Y'])):
            x = xy[0] + [0]*(max(numVars-len(xy[0]),0)) # padding
            y = [xy[1]] if type(xy[1]) == float or type(xy[1]) == int else xy[1]
            y = y + [0]*(max(numYs-len(y),0)) # padding
            p = x+y # because it is only one point 
            p = torch.tensor(p)
            #replace nan and inf
            p = torch.nan_to_num(p, nan=0.0, 
                                 posinf=self.threshold[1], 
                                 neginf=self.threshold[0])
            p[p>self.threshold[1]] = self.threshold[1] # clip the upper bound
            p[p<self.threshold[0]] = self.threshold[0] # clip the lower bound
            points[:,idx] = p
        
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        inputs = torch.tensor(inputs, dtype=torch.long)
        outputs = torch.tensor(outputs, dtype=torch.long)
        return inputs, outputs, points


In [5]:
import json
from tqdm import tqdm
import glob
def processDataFiles(path, files):
    text = ''""
    for f in tqdm(files):
        with open(path+f, 'r') as h: 
            lines = h.read() # don't worry we won't run out of file handles
            if lines[-1]==-1:
                lines = lines[:-1]
            text += lines #json.loads(line)       
            
    return text

In [6]:
path = '{}/{}/Train/'.format(dataDir, dataFolder)
files = os.listdir(path)[:maxNumFiles]
text = processDataFiles(path, files)
chars = sorted(list(set(text))+['_','T','<','>']) # extract unique characters from the text before converting the text to a list
# T is for the test data
text = text.split('\n') # convert the raw text to a set of examples
text = text[:-1] if len(text[-1]) == 0 else text
random.shuffle(text) # shuffle the dataset, it's important for combined number of variables
train_dataset = CharDataset(text, blockSize, chars, target=target) 

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:14<00:00,  2.13it/s]


data has 300000 examples, 48 unique.


In [7]:
idx = np.random.randint(train_dataset.__len__())
inputs, outputs, points = train_dataset.__getitem__(idx)
print('inputs:{}'.format(inputs))
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\ninputs:{}\noutputs:{}\npoints:{}'.format(idx,inputs,outputs,points))
minimum, maximum = points.min(), points.max()
print(minimum, maximum)

inputs:tensor([21, 43, 36, 39,  3, 23,  5, 45, 13,  6, 23,  4, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32])
id:121958
inputs:<sin(C*x3+C)________________________________________________________________________________________
outputs:sin(C*x3+C)>________________________________________________________________________________________
points:tensor([[ 1.0000,  0.2400,  1.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.4200,  0.7200,  1.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.0000,  1.0000,  0.2500,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.0000,  1.0000,  1.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2800,  1.0000,  1.

In [8]:
path = '{}/{}/Val/'.format(dataDir,dataFolder)
files = os.listdir(path)[:maxNumFiles]
textVal = processDataFiles(path, files)
textVal = textVal.split('\n') # convert the raw text to a set of examples
val_dataset = CharDataset(textVal, blockSize, chars, target=target)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 29.08it/s]


data has 1001 examples, 48 unique.


In [9]:
idx = np.random.randint(val_dataset.__len__())
inputs, outputs, points = val_dataset.__getitem__(idx)
print(points.min(), points.max())
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\ninputs:{}\noutputs:{}\npoints:{}'.format(idx,inputs,outputs,points))

tensor(0.) tensor(1.)
id:435
inputs:<C*x2+C_____________________________________________________________________________________________
outputs:C*x2+C>_____________________________________________________________________________________________
points:tensor([[0.2800, 1.0000, 0.4200,  ..., 0.0000, 0.0000, 0.0000],
        [1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        [1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000]])


In [10]:
path = '{}/{}/Test/'.format(dataDir,dataFolder)
files = os.listdir(path)[:maxNumFiles]
textTest = processDataFiles(path, files)
textTest = textTest.split('\n') # convert the raw text to a set of examples
# test_dataset_target = CharDataset(textTest, blockSize, chars, target=target)
test_dataset = CharDataset(textTest, blockSize, chars)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 27.25it/s]

data has 1001 examples, 48 unique.


In [11]:
idx = np.random.randint(test_dataset.__len__())
inputs, outputs, points = test_dataset.__getitem__(idx)
print(points.min(), points.max())
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\ninputs:{}\noutputs:{}\npoints:{}'.format(idx,inputs,outputs,points))

tensor(0.) tensor(1.)
id:860
inputs:<0.14*x3+sqrt(abs(-0.93*x5))________________________________________________________________________
outputs:0.14*x3+sqrt(abs(-0.93*x5))>________________________________________________________________________
points:tensor([[1.0000, 1.0000, 0.6400,  ..., 0.0000, 0.0000, 0.0000],
        [0.7300, 1.0000, 0.6500,  ..., 0.0000, 0.0000, 0.0000],
        [1.0000, 1.0000, 0.5200,  ..., 0.0000, 0.0000, 0.0000],
        [1.0000, 0.9100, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.3600, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.7300, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000]])


In [12]:
# load the general 1-5 model
from mingpt.model import GPT, GPTConfig, PointNetConfig
pconf = PointNetConfig(embeddingSize=embeddingSize, 
                       numberofPoints=numPoints, 
                       numberofVars=numVars, 
                       numberofYs=numYs)
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=embeddingSize, padding_idx=train_dataset.paddingID)
model = GPT(mconf, pconf)

In [13]:
# load the best model
# fName = '{}_SymbolicGPT_{}_{}_{}_MINIMIZE.txt'.format(dataInfo, 
#                                              'GPT_PT_Summation', 
#                                              'Padding',
#                                              blockSize)
# {
# print(fName)
fName = ckpt
ckptPath = '{}/{}.pt'.format(addr,ckpt)
model.load_state_dict(torch.load(ckptPath))
model = model.eval()#.cuda()

In [14]:
# add a safe wrapper for numpy math functions
from numpy import *
import numpy as np

def divide(x, y):
  x = np.nan_to_num(x)
  y = np.nan_to_num(y)
  return np.divide(x,y+1e-5)

def sqrt(x):
  x = np.nan_to_num(x)
  return np.sqrt(np.abs(x)) 

# Mean square error
def mse(y, y_hat):
    y_hat = np.reshape(y_hat, [1, -1])[0]
    y_gold = np.reshape(y, [1, -1])[0]
    our_sum = 0
    for i in range(len(y_gold)):
        our_sum += (y_hat[i] - y_gold[i]) ** 2

    return our_sum / len(y_gold)

# Mean square error
def relativeErr(y, y_hat):
    y_hat = np.reshape(y_hat, [1, -1])[0]
    y_gold = np.reshape(y, [1, -1])[0]
    our_sum = 0
    for i in range(len(y_gold)):
        if y_gold[i] < 1: 
            # use regular MSE
            our_sum += (y_hat[i] - y_gold[i]) ** 2
        else:
            # use relative MSE
            our_sum += ((y_hat[i] - y_gold[i])/y_gold[i]) ** 2

    return our_sum / len(y_gold)

def lossFunc(constants, eq, X, Y):
    err = 0
    eq = eq.replace('C','{}').format(*constants)

    for x,y in zip(X,Y):
        eqTemp = eq + ''
        for i,e in enumerate(x):
            eqTemp = eqTemp.replace('x{}'.format(i+1), str(e))
        try:
            yHat = eval(eqTemp)
        except:
            yHat = 100
        err += (y-yHat)**2
    err /= len(Y)
    return err

In [ ]:
# alright, let's sample some character-level symbolic GPT
from mingpt.utils import sample
from gp_model import Genetic_Model
from mlp_model import MLP_Model
from scipy.optimize import least_squares, newton, minimize

numSamples = 100
cutoff = 0.01
step = 10

path = '{}/{}/Test/'.format(dataDir,dataFolder)
files = os.listdir(path)[:maxNumFiles]
textTest = processDataFiles(path, files)
textTest = textTest.split('\n') # convert the raw text to a set of examples
textTest = textTest[:numSamples] + textTest[-numSamples:] #[-numSamples:-2]
test_dataset = CharDataset(textTest, blockSize, chars)
loader = torch.utils.data.DataLoader(
                                test_dataset, 
                                shuffle=False, 
                                pin_memory=True,
                                batch_size=1,
                                num_workers=0)


xR = [5, 10, 20, 50, 100, 250, 500, 1000, 5000] #+ range(1,numPoints,step)
try:
    successfullPredictionPercentage = {}
    for pNum in tqdm(xR):
        resultDict = {}

        modelName = 'SymbolicGPT'
        resultDict[fName] = {modelName:[],
                             'GP':[],
                             'MLP':[]}

        numEquationsLessThanCutoff = 0
        for i, batch in enumerate(loader):

            inputs,outputs,points = batch

            print('Test Case {}.'.format(i))

            t = json.loads(textTest[i])

            inputs = inputs[:,0:1]#.cuda()
            points = points[:,:,:pNum]#.cuda()
            outputsHat = sample(model, inputs, blockSize, points=points,
                          temperature=1.0, sample=True, 
                          top_k=40)[0]

            # filter out predicted
            target = ''.join([train_dataset.itos[int(i)] for i in outputs[0]])
            predicted = ''.join([train_dataset.itos[int(i)] for i in outputsHat])

            target = target.strip(train_dataset.paddingToken).split('>')
            target = target[0] if len(target[0])>=1 else target[1]
            target = target.strip('<').strip(">")
            predicted = predicted.strip(train_dataset.paddingToken).split('>')
            predicted = predicted[0] if len(predicted[0])>=1 else predicted[1]
            predicted = predicted.strip('<').strip(">")

            print('Target:{}\nSkeleton:{}'.format(target, predicted))

            # train a regressor to find the constants (too slow)
            c = [1 for i,x in enumerate(predicted) if x=='C']            

            try:
                if len(c) == 0:
                    pass # do nothing
                else:

                    # for easier comparison, we are using minimize package  
                    cHat = minimize(lossFunc, c,
                                   args=(predicted, t['X'], t['Y'])) 
                    predicted = predicted.replace('C','{}').format(*cHat.x)
            except:
                print('Wrong Equation:{}'.format(predicted))
                raise
                predicted = 0

            # TODO: let's enjoy GPU

            print('Skeleton+LS:{}'.format(predicted))

            Ys = [] #t['YT']
            Yhats = []
            for xs in t['XT']:
                try:
                    eqTmp = target + '' # copy eq
                    eqTmp = eqTmp.replace(' ','')
                    eqTmp = eqTmp.replace('\n','')
                    for i,x in enumerate(xs):
                        # replace xi with the value in the eq
                        eqTmp = eqTmp.replace('x{}'.format(i+1), str(x))
                        if ',' in eqTmp:
                            assert 'There is a , in the equation!'
                    YEval = eval(eqTmp)
                    YEval = 0 if np.isnan(YEval) else YEval
                    YEval = 100 if np.isinf(YEval) else YEval
                except:
                    YEval = 100 #TODO: Maybe I have to punish the model for each wrong template not for each point
                Ys.append(YEval)
                try:
                    eqTmp = predicted + '' # copy eq
                    eqTmp = eqTmp.replace(' ','')
                    eqTmp = eqTmp.replace('\n','')
                    for i,x in enumerate(xs):
                        # replace xi with the value in the eq
                        eqTmp = eqTmp.replace('x{}'.format(i+1), str(x))
                        if ',' in eqTmp:
                            assert 'There is a , in the equation!'
                    Yhat = eval(eqTmp)
                    Yhat = 0 if np.isnan(Yhat) else Yhat
                    Yhat = 1000 if np.isinf(Yhat) else Yhat
                except:
                    Yhat = 1000
                Yhats.append(Yhat)
            err = relativeErr(Ys,Yhats)
            print('Err:{}'.format(err))

            if type(err) is np.complex128 or np.complex:
                err = abs(err.real)

            resultDict[fName][modelName].append(err)

            if err < cutoff:
                numEquationsLessThanCutoff += 1

            print('') # just an empty line

        successfullPredictionPercentage[pNum] = numEquationsLessThanCutoff/numSamples
except KeyboardInterrupt:
    print('KeyboardInterrupt')

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]

data has 200 examples, 48 unique.
Test Case 0.
Target:x1-sin(0.64*x1)
Skeleton:sin(C**2*x1**2+2*C**2*x1+C**2)
Skeleton+LS:sin(-3.7110585590876637e-06**2*x1**2+2*0.5698999171172695**2*x1+3.5242469879475755**2)
Err:0.7926030813435517

Test Case 1.
Target:sin(1.27*x1+0.45)
Skeleton:sqrt(abs(C*x1+C))*sin(C*x1+C)
Skeleton+LS:sqrt(abs(-0.0009445014883192617*x1+1.0010663563027986))*sin(1.2689571930684664*x1+0.45363369549192767)
Err:2.6194182324486326e-06

Test Case 2.
Target:-0.33*x1**2*sin(1.0*x1)-0.63*x1
Skeleton:sin(C**2*x1**2+2*C**2*x1+C**2)
Skeleton+LS:sin(1.3156846869809977**2*x1**2+2*1.100105075956499**2*x1+1.638689574880538**2)
Err:1.2378204160552089

Test Case 3.
Target:1.21*x1+sqrt(abs(0.22-0.58*x1))-0.77
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*-0.28903168650965666*x1+2*4.624516667970811)
Err:0.9001986511573654

Test Case 4.
Target:-0.57*x1*sin(0.28*x1)+0.09
Skeleton:sin(C*x1+C)
Skeleton+LS:sin(0.5188591844286504*x1+2.8941369248999207)
Err:3.2481057726054163

Test Case 5.
Target:

Target:1.98*x1+sqrt(abs(x1))+0.28
Skeleton:sin(C**2*x1**2+2*C**2*x1+C**2)
Skeleton+LS:sin(-0.009472177740661348**2*x1**2+2*0.013137011405375437**2*x1+1.2531973071861224**2)
Err:0.8293113196232607

Test Case 50.
Target:-0.21*x1+sqrt(abs(x1))-0.7
Skeleton:sin(sin(C*x1+C))
Skeleton+LS:sin(sin(0.2714002812060171*x1+-0.25302335822559824))
Err:0.06776892404658247

Test Case 51.
Target:1.58*x1-1.14
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*0.15543040405106465*x1+2*3.533108539957254)
Err:0.7353705587692921

Test Case 52.
Target:2*x1+0.28
Skeleton:sin(C**2*x1**2+2*C**2*x1+C**2)
Skeleton+LS:sin(1.3407227561935149**2*x1**2+2*-6.0032135858798004e-06**2*x1+0.3799379447593586**2)
Err:1.0021621428181153

Test Case 53.
Target:-sin(0.07*x1)*sin(x1)
Skeleton:sin(C*x1+C)
Skeleton+LS:sin(0.0323838592578153*x1+3.164958077209904)
Err:0.1705192922940724

Test Case 54.
Target:x1-0.88*sin(x1-0.82)
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*1.1778891229767114*x1+2*1.087345028513697)
Err:1.1303474558592528

Tes

Target:-sin(0.33*x1+0.32)
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*0.16595841136496353*x1+2*1.7296810888307508)
Err:8.664465653754278e-06

Test Case 98.
Target:x1+0.41*sqrt(abs(0.34*x1))
Skeleton:sin(C*x1+C)**2
Skeleton+LS:sin(1.941892601777928*x1+2.7143094642681973)**2
Err:0.8338658208510964

Test Case 99.
Target:x1*sqrt(abs(x1))
Skeleton:sin(C**2*x1**2+2*C**2*x1+C**2)
Skeleton+LS:sin(-0.5904702582257888**2*x1**2+2*0.8950488856855566**2*x1+-0.45603311376485794**2)
Err:0.9592224318731175

Test Case 100.
Target:sin(0.95*x1**2+0.34*x1)
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*-0.06597579566205936*x1+2*0.23670135410560575)
Err:0.553119732985752

Test Case 101.
Target:sin(0.14*x2*x3)
Skeleton:sin(sin(C*x2+C))
Skeleton+LS:sin(sin(0.2192172684361261*x2+-0.020168903769099165))
Err:0.5878656294451676

Test Case 102.
Target:-0.83*x2-0.11
Skeleton:2*C*x1+2*C
Skeleton+LS:2*-0.016955162283555477*x1+2*-0.6939304402645511
Err:5.747080440552751

Test Case 103.
Target:sin(sqrt(abs(x2)))
Skeleton:s

Target:sqrt(abs(-0.16*x3-0.38))
Skeleton:sqrt(abs(sin(C*x1+C)))
Skeleton+LS:sqrt(abs(sin(1.7124683961554075*x1+0.47919501175861984)))
Err:0.15525435494262235

Test Case 150.
Target:sqrt(abs(sqrt(abs(0.55*x2+0.43))))
Skeleton:sin(sin(C*x2+C))
Skeleton+LS:sin(sin(-1.7668908768284943e-07*x2+1.5707956458394232))
Err:0.12494483883686006

Test Case 151.
Target:-0.27*x1*x4+1.17*x1-0.18*x3-0.83*x4-0.5
Skeleton:C*x1+C
Skeleton+LS:0.8216687336815214*x1+-2.1872169567997535
Err:51.13525201222437

Test Case 152.
Target:0.79*x2-0.42
Skeleton:sin(C*x1+C)
Skeleton+LS:sin(0.04621909993910045*x1+0.9790604384423892)
Err:0.5045424428383213

Test Case 153.
Target:sqrt(abs(x1))*sqrt(abs(x4-0.07))
Skeleton:sin(sin(C*x1+C))
Skeleton+LS:sin(sin(0.3183248253521592*x1+0.8424363441866314))
Err:0.7205213948214202

Test Case 154.
Target:sin(0.43*x4+0.29)
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*2.6366492876010565*x1+2*0.06883965499222604)
Err:1.2472765348843053

Test Case 155.
Target:sqrt(abs(x4+x5-0.5))
Skeleton

...

 67%|███████████████████████████████████████████████████████▎                           | 6/9 [59:37<31:54, 638.25s/it]

Target:-sin(0.18*x5**2+0.18*x5)
Skeleton:sin(C*x1+C)
Skeleton+LS:sin(0.014581133929271496*x1+3.69854305482955)
Err:0.961617365239249

Test Case 0.
Target:x1-sin(0.64*x1)
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*-0.32478702563974426*x1+2*4.785418755774995)
Err:0.7926045339226833

Test Case 1.
Target:sin(1.27*x1+0.45)
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*0.6343013890487684*x1+2*0.22712281746381421)
Err:2.2623470731148046e-06

Test Case 2.
Target:-0.33*x1**2*sin(1.0*x1)-0.63*x1
Skeleton:sin(C*x1+C)
Skeleton+LS:sin(0.15060817163422346*x1+2.8968264371117924)
Err:1.2473242488714844

Test Case 3.
Target:1.21*x1+sqrt(abs(0.22-0.58*x1))-0.77
Skeleton:sin(C*x1+C)
Skeleton+LS:sin(0.5780612088483692*x1+0.17575078851549894)
Err:0.9001976485725577

Test Case 4.
Target:-0.57*x1*sin(0.28*x1)+0.09
Skeleton:sin(C*x1+C)
Skeleton+LS:sin(0.5188591844286504*x1+2.8941369248999207)
Err:3.2481057726054163

Test Case 5.
Target:0.04*x1**2+0.17*x1+0.2*sin(0.15*x1)
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(

Target:sin(sqrt(abs(x1-0.69)))
Skeleton:C**2*x1**2+2*C**2*x1+C**2+sin(C*x1+C)
Skeleton+LS:0.18978185055165117**2*x1**2+2*0.2608004688381524**2*x1+0.04055512638458569**2+sin(0.002326874839078373*x1+2.7196007327982925)
Err:0.9862048061961403

Test Case 49.
Target:1.98*x1+sqrt(abs(x1))+0.28
Skeleton:sqrt(abs(C*x1+C))*sin(C*x1+C)
Skeleton+LS:sqrt(abs(25.172775199175632*x1+0.3505312403376704))*sin(-0.2222490268018199*x1+2.6675372676035414)
Err:0.00866502814144379

Test Case 50.
Target:-0.21*x1+sqrt(abs(x1))-0.7
Skeleton:sin(C*x1+C)
Skeleton+LS:sin(-3.7889391005945856*x1+-1.6380705374040676)
Err:0.789719557797021

Test Case 51.
Target:1.58*x1-1.14
Skeleton:sqrt(abs(C*x1+C))+sin(C*x1+C)
Skeleton+LS:sqrt(abs(49.349595213986674*x1+-23.99486978388571))+sin(1.3507748814157234*x1+-2.608337972651798)
Err:2.148672457097878

Test Case 52.
Target:2*x1+0.28
Skeleton:sin(C*x1+C)
Skeleton+LS:sin(1.0941681352309035e-06*x1+1.5707938781288668)
Err:0.7905871082310828

Test Case 53.
Target:-sin(0.07*x1)*sin(x

Target:-sin(0.33*x1+0.32)
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*0.16595841136496353*x1+2*1.7296810888307508)
Err:8.664465653754278e-06

Test Case 98.
Target:x1+0.41*sqrt(abs(0.34*x1))
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*0.2579218169524446*x1+2*3.2908057436995746)
Err:0.8198444948045935

Test Case 99.
Target:x1*sqrt(abs(x1))
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*-0.20184484879829465*x1+2*4.42667503860859)
Err:0.8461254799268739

Test Case 100.
Target:sin(0.95*x1**2+0.34*x1)
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*-0.06597579566205936*x1+2*0.23670135410560575)
Err:0.553119732985752

Test Case 101.
Target:sin(0.14*x2*x3)
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*1.338053204497608*x1+2*1.0036962533066847)
Err:1.0433308799905097

Test Case 102.
Target:-0.83*x2-0.11
Skeleton:sin(C*x1+C)
Skeleton+LS:sin(-6.490799773263092e-07*x1+4.712390314654434)
Err:8.523403915998202

Test Case 103.
Target:sin(sqrt(abs(x2)))
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*1.5892112109841863*

Target:0.91*x4+sqrt(abs(-0.93*x4-0.92))+0.84
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*1.5614847994319967*x1+2*-0.07386793988743871)
Err:1.0655672640316003

Test Case 149.
Target:sqrt(abs(-0.16*x3-0.38))
Skeleton:sqrt(abs(sin(C*x1+C)))
Skeleton+LS:sqrt(abs(sin(1.7124683961554075*x1+0.47919501175861984)))
Err:0.15525435494262235

Test Case 150.
Target:sqrt(abs(sqrt(abs(0.55*x2+0.43))))
Skeleton:sin(C*x1+C)
Skeleton+LS:sin(-1.8569567459110969e-06*x1+1.5707874406104625)
Err:0.054082922836294615

Test Case 151.
Target:-0.27*x1*x4+1.17*x1-0.18*x3-0.83*x4-0.5
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*0.29897181960355007*x1+2*2.2627965268325996)
Err:39.62176333728911

Test Case 152.
Target:0.79*x2-0.42
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*1.4784004376819744*x1+2*0.35847152811668614)
Err:1.2653170180088793

Test Case 153.
Target:sqrt(abs(x1))*sqrt(abs(x4-0.07))
Skeleton:sin(2*C*x1+2*C)
Skeleton+LS:sin(2*1.641799893954669*x1+2*-0.20649512210675844)
Err:1.1353310132831236

Test Case 154.

In [ ]:
from matplotlib import pyplot as plt
linestyles = ["-","dashdot","dotted","--"]
plt.plot(xR, 
       list(successfullPredictionPercentage.values()), 
       linestyle=linestyles[0], 
       label='SymbolicGPT')

plt.legend(loc="upper left")
plt.title(", Model: 1-5 Variables")
plt.xlabel("Number of Points")
plt.ylabel("% Prediction With Relative Err < {}".format(cutoff))


# name = '{}.png'.format(fName.split('.txt')[0])
# print(name)
# plt.savefig(name)

In [ ]:
successfullPredictionPercentage